In [30]:
# 经典得分函数

class getS:
    def __init__(self, IVq, q):

        self.data = IVq
        self.q = q
        self.list_function = [self.s]

    def getScore(self, numb=0):
        try:
            return self.list_function[numb]()
        except:
            print("下标索引的距离公式不存在")
            return self.list_function[0]()

    def s(self):

        a = self.data[0][0] ** self.q
        b = self.data[0][1] ** self.q
        c = self.data[1][0] ** self.q
        d = self.data[1][1] ** self.q

        return (a + b - c - d) / 2



In [56]:
"""
用得分实现数值转换计算属性权重
"""
import numpy as np
import pandas as pd
def CRITIC(data,q):
    res=[]
    for enmu in data:
        res1=[]
        for enmu1 in enmu:
            a = enmu1[0][0] ** q
            b = enmu1[0][1] ** q
            c = enmu1[1][0] ** q
            d = enmu1[1][1] ** q
            res1.append((a + b - c - d) / 2)
        res.append(res1)
    res=np.array(res)
    correlation_matrix = np.corrcoef(res, rowvar=False)
    n_correlation_matrix,m_correlation_matrix=correlation_matrix.shape
    display(n_correlation_matrix,m_correlation_matrix)
    data_final=np.array(res)
    n_data, m_data = data_final.shape
    """
    归一化矩阵列平均
    """
    means = np.mean(res, axis=0)
    # 初始化标准差数组
    r = np.zeros(m_data)
    # 计算每个属性的标准差 r_j
    for j in range(m_data):
        # 计算 r_j
        r[j] = np.sqrt(np.sum((data_final[:, j] - means[j]) ** 2) / (n_data - 1))
    """
    关联矩阵行矩阵
    """
    correlation_matrix_means=np.mean(correlation_matrix,axis=1)
    g_list=[]
    for enum in correlation_matrix_means:
        g_list.append(m_correlation_matrix*(1-enum))
    C=[]
    for i in range(len(r)):
        C.append(r[i]*g_list[i])
    value=[enum/sum(C) for enum in C]
    return value


In [53]:
"""
用得分实现数值转换计算属性权重,与论文验证对错
"""
import numpy as np
import pandas as pd

def CRITIC1(datag):
    res=datag
    display(res)
    correlation_matrix = np.corrcoef(res, rowvar=False)
    n_correlation_matrix,m_correlation_matrix=correlation_matrix.shape
    display(n_correlation_matrix,m_correlation_matrix)
    data=np.array(res)
    n_data, m_data = data.shape
    """
    归一化矩阵列平均
    """
    means = np.mean(res, axis=0)
    # 初始化标准差数组
    r = np.zeros(m_data)
    # 计算每个属性的标准差 r_j
    for j in range(m_data):
        # 计算 r_j
        r[j] = np.sqrt(np.sum((data[:, j] - means[j]) ** 2) / (n_data - 1))
    """
    关联矩阵行矩阵
    """
    correlation_matrix_means=np.mean(correlation_matrix,axis=1)
    g_list=[]
    for enum in correlation_matrix_means:
        g_list.append(m_correlation_matrix*(1-enum))
    C=[]
    for i in range(len(r)):
        C.append(r[i]*g_list[i])
    value=[enum/sum(C) for enum in C]

    return value

array([[1.   , 1.   , 0.   , 0.667],
       [0.741, 0.684, 0.   , 0.   ],
       [0.926, 0.526, 0.5  , 1.   ],
       [0.   , 0.   , 1.   , 0.167]])

4

4

[0.19421309360194777,
 0.2078697783991343,
 0.4191604704652144,
 0.17875665753370337]

In [33]:
from math import pow, fabs


class getD:
    def __init__(self,IVQ1,IVQ2,q):
        '''

        :param IVQ1:
        :param IVQ2:
        :param q:
        '''
        self.IVQ1=IVQ1
        self.IVQ2=IVQ2
        self.q=q
        self.list_function=[self.func1]

    def getDistance(self,numb=0):
        '''
        function: 选取距离公式
        :return: 距离公式的计算结果
        '''
        try:
            return self.list_function[numb]()
        except:
            print("下标索引的距离公式不存在")
            return self.list_function[0]()


    def get_pi(self, a, *waste1, **waste2):
        """
            function : 求犹豫度 —— 求 d 即 get_d 中会用到
            a        : 一个模糊集, 形式:（[x,y]，[x,y]）
            *waste1  : 回收多余的输入参数
            **waste2 : 回收多余的输入键值对参数
            return   : 犹豫度列表，形式： [x,y]
        """
        pi_f = pow((1 - pow(a[0][1], self.q) - pow(a[1][1], self.q)), 1 / self.q)
        pi_r = pow((1 - pow(a[0][0], self.q) - pow(a[1][0], self.q)), 1 / self.q)
        return [pi_f, pi_r]

    def func1(self, *waste1, **waste2):
        """
            function : 求用于计算支撑度中用到的 d
            a1       : 一个模糊集, 形式:（[x,y]，[x,y]）
            a2       : 一个模糊集, 形式:（[x,y]，[x,y]）
            *waste1  : 回收多余的输入参数
            **waste2 : 回收多余的输入键值对参数
            return   : d 的值
        """
        # 提取隶属区间的上下界、非隶属区间的上下界限、犹豫度的上下界限
        a1=self.IVQ1
        a2=self.IVQ2
        u_1_f, u_1_r, v_1_f, v_1_r, pi_1_f, pi_1_r = *a1[0], *a1[1], *self.get_pi(a1)
        u_2_f, u_2_r, v_2_f, v_2_r, pi_2_f, pi_2_r = *a2[0], *a2[1], *self.get_pi(a2)

        # 将公式分为三个部分：隶属区间u、费隶属区间v、犹豫度π
        part_u = fabs(u_1_f ** self.q - u_2_f ** self.q) + fabs(u_1_r ** self.q - u_2_r ** self.q)
        part_v = fabs(v_1_f ** self.q - v_2_f ** self.q) + fabs(v_1_r ** self.q - v_2_r ** self.q)
        part_r = fabs(pi_1_f ** self.q - pi_2_f ** self.q) + fabs(pi_1_r ** self.q - pi_2_r ** self.q)

        # 整合三个部分的公式
        d_value = 1 / 4 * (part_u + part_v + part_r)
        return d_value


In [34]:
class Operator_IVQ_ROFS:
    """
        IVQ_ROFS 区间值广义正交模糊集 类
    """

    def __init__(self, data_list=[], weight_list=[], q=3, x=2, a=0.3, b=0.5):
        '''
        function: 初始化类的初始值, 具有默认初始值, 和检查数据列表和权重列表维度匹配的功能
        :param data_list: 区间值广义正交模糊向量
        :param weight_list: 权重向量
        :param q: 参数q
        :param x: 参数x
        :param a: 参数a
        :param b: 参数b

        '''

        if len(data_list):
            self.data_list = data_list  # 初始化数据列表
        else:   # 默认初始为一个 5 个模糊数的列表
            self.data_list = [([0.85, 0.95], [0.1, 0.2]), ([0.8, 0.9], [0.1, 0.2]), ([0.85, 0.95], [0.1, 0.2]), ([0.7, 0.8], [0.2, 0.3]),
        ([0.4, 0.5], [0.5, 0.6])]

        if len(weight_list):
            self.weight_list = weight_list  # 初始化权重列表
        else:   # 默认值设为相等
            self.weight_list = [1/len(self.data_list) for i in range(len(self.data_list))]

        # 如果存在数据和权重列表不匹配的情况, 则做切片处理
        if len(self.data_list) != len(self.weight_list):
            min_len = min(len(self.data_list), len(self.weight_list))
            self.data_list = self.data_list[::min_len]
            self.weight_list = self.weight_list[:min_len]

        self.getdis=getD  # 初始化距离公式函数
        self.getsco=getS  # 初始化得分函数

        self.q = q  # 初始化参数 q
        self.x = x  # 初始化参数 x
        self.a = a  # 初始化参数 a
        self.b = b  # 初始化参数 b



    # set function
    def set_data(self, data, *waste1, **waste2):
        """
            funciton    : 修改 datalist
            data        : 数据集, 单个数据形式:（[x,y]，[x,y]）
            return      : 无返回值
        """
        self.data_list = data  # 初始化数据列表

    def set_q(self, q, *waste1, **waste2):
        """
            funciton    : 修改 q parameter
            q           : 实数
            return      : 无返回值
        """
        self.q = q  # 初始化数据列表

    def set_x(self, x, *waste1, **waste2):
        """
            funciton    : 修改 x parameter
            x           : 实数
            return      : 无返回值
        """
        self.x = x  # 初始化数据列表

    def set_a(self, a, *waste1, **waste2):
        """
            funciton    : 修改 y parameter
            a           : 实数
            return      : 无返回值
        """
        self.a = a  # 初始化数据列表

    def set_b(self, b, *waste1, **waste2):
        """
            funciton    : 修改 z parameter
            b           : 实数
            return      : 无返回值
        """
        self.b = b # 初始化数据列表

    def set_weight(self, weight, *waste1, **waste2):
        """
            funciton    : 修改 weight
            weigh       : 权重想来给你
            return      : 无返回值
        """
        self.weight_list = weight

    # 基础运算法则相关
    def is_include(self, a1, a2, *waste1, **waste2):
        """
            funciton    : 判断包含关系
            a1          : 一个模糊集, 形式:（[x,y]，[x,y]）
            a2          : 一个模糊集, 形式:（[x,y]，[x,y]）
            return      : 布尔值
        """
        u_1_f, u_1_r, v_1_f, v_1_r = *a1[0], *a1[1]
        u_2_f, u_2_r, v_2_f, v_2_r = *a2[0], *a2[1]
        if u_1_f <= u_2_f and u_1_r <= u_2_r and v_1_f >= v_2_f and v_1_r >= v_2_r:
            return True
        return False

    def is_equal(self, a1, a2, *waste1, **waste2):
        """
            funciton    : 判断相等关系
            a1          : 一个模糊集, 形式:（[x,y]，[x,y]）
            a2          : 一个模糊集, 形式:（[x,y]，[x,y]）
            return      : 布尔值
        """
        u_1_f, u_1_r, v_1_f, v_1_r = *a1[0], *a1[1]
        u_2_f, u_2_r, v_2_f, v_2_r = *a2[0], *a2[1]
        if u_1_f == u_2_f and u_1_r == u_2_r and v_1_f == v_2_f and v_1_r == v_2_r:
            return True
        return False

    def is_legal(self, a, q=0, *waste1, **waste2):
        """
            funciton    : 判断模糊集是否符合广义正交的条件
            a           : 一个模糊集, 形式:（[x,y]，[x,y]）
            return      : 布尔值
        """
        if q < 1:
            q = self.q
        u_f, u_r, v_f, v_r = *a[0], *a[1]
        if (u_r ** q + v_r ** q) > 1:
            return False
        return True

    def complement(self, a, *waste1, **waste2):
        """
            function : 区间值广义正交模糊集的 补 运算
            a        : 一个模糊集, 形式:（[x,y]，[x,y]）
            return   : 补集, 形式:（[x,y]，[x,y]）
        """
        u_f, u_r, v_f, v_r = *a[0], *a[1]
        return ([v_f, v_r], [u_f, u_r])

    def intersect(self, a1, a2, *waste1, **waste2):
        """
            funciton    : 区间值广义正交模糊集的 交 运算
            a1          : 一个模糊集, 形式:（[x,y]，[x,y]）
            a2          : 一个模糊集, 形式:（[x,y]，[x,y]）
            return      : 交集, 形式:（[x,y]，[x,y]）
        """
        u_1_f, u_1_r, v_1_f, v_1_r = *a1[0], *a1[1]
        u_2_f, u_2_r, v_2_f, v_2_r = *a2[0], *a2[1]
        return ([min(u_1_f, u_2_f), min(u_1_r, u_2_r)],
                [max(v_1_f, v_2_f), max(v_1_r, v_2_r)])

    def unite(self, a1, a2, *waste1, **waste2):
        """
            funciton    : 区间值广义正交模糊集的 并 运算
            a1          : 一个模糊集, 形式:（[x,y]，[x,y]）
            a2          : 一个模糊集, 形式:（[x,y]，[x,y]）
            return      : 并集, 形式:（[x,y]，[x,y]）
        """
        u_1_f, u_1_r, v_1_f, v_1_r = *a1[0], *a1[1]
        u_2_f, u_2_r, v_2_f, v_2_r = *a2[0], *a2[1]
        return ([max(u_1_f, u_2_f), max(u_1_r, u_2_r)],
                [min(v_1_f, v_2_f), min(v_1_r, v_2_r)])

    def add(self, a1, a2, q=0, *waste1, **waste2):
        """
            funciton    : 区间值广义正交模糊集的 和 运算
            a1          : 一个模糊集, 形式:（[x,y]，[x,y]）
            a2          : 一个模糊集, 形式:（[x,y]，[x,y]）
            return      : 加法结果, 形式:（[x,y]，[x,y]）
        """
        u_1_f, u_1_r, v_1_f, v_1_r = *a1[0], *a1[1]
        u_2_f, u_2_r, v_2_f, v_2_r = *a2[0], *a2[1]
        if q <= 0:
            q = self.q
        u_f = (u_1_f ** q + u_2_f ** q - (u_1_f ** q) * (u_2_f ** q)) ** (1 / q)
        u_r = (u_1_r ** q + u_2_r ** q - (u_1_r ** q) * (u_2_r ** q)) ** (1 / q)
        v_f = (v_1_f) * (v_2_f)
        v_r = (v_1_r) * (v_2_r)
        return ([u_f, u_r], [v_f, v_r])

    def multi(self, a1, a2, q=0, *waste1, **waste2):
        """
            funciton    : 区间值广义正交模糊集的 积 运算
            a1          : 一个模糊集, 形式:（[x,y]，[x,y]）
            a2          : 一个模糊集, 形式:（[x,y]，[x,y]）
            return      : 乘法结果, 形式:（[x,y]，[x,y]）
        """
        u_1_f, u_1_r, v_1_f, v_1_r = *a1[0], *a1[1]
        u_2_f, u_2_r, v_2_f, v_2_r = *a2[0], *a2[1]
        if q <= 0:
            q = self.q
        u_f = (u_1_f) * (u_2_f)
        u_r = (u_1_r) * (u_2_r)
        v_f = (v_1_f ** q + v_2_f ** q - (v_1_f ** q) * (v_2_f ** q)) ** (1 / q)
        v_r = (v_1_r ** q + v_2_r ** q - (v_1_r ** q) * (v_2_r ** q)) ** (1 / q)
        return ([u_f, u_r], [v_f, v_r])

    def kmulti(self, a, k, q=0, *waste1, **waste2):
        """
            function : 区间值广义正交模糊集的 数乘 运算
            a        : 一个模糊集, 形式:（[x,y]，[x,y]）
            k        : ka 的常数 k
            return   : 数乘 运算法则 的值，形式：（[x,y]，[x,y]）
        """
        u_a_f, u_a_r, v_a_f, v_a_r = *a[0], *a[1]
        if q <= 0:
            q = self.q
        u_f = (1 - (1 - (u_a_f) ** q) ** k) ** (1 / q)
        u_r = (1 - (1 - (u_a_r) ** q) ** k) ** (1 / q)
        v_f = (v_a_f) ** k
        v_r = (v_a_r) ** k
        return ([u_f, u_r], [v_f, v_r])

    def pow(self, a, k, q=0, *waste1, **waste2):
        """
            function : 区间值广义正交模糊集的 乘方 运算
            a        : 一个模糊集, 形式:（[x,y]，[x,y]）
            k        : ka 的常数 k
            return   : 乘方 运算法则 的值，形式：（[x,y]，[x,y]）
        """
        u_a_f, u_a_r, v_a_f, v_a_r = *a[0], *a[1]
        if q <= 0:
            q = self.q
        v_f = (1 - (1 - (v_a_f) ** q) ** k) ** (1 / q)
        v_r = (1 - (1 - (v_a_r) ** q) ** k) ** (1 / q)
        u_f = (u_a_f) ** k
        u_r = (u_a_r) ** k
        return ([u_f, u_r], [v_f, v_r])

    def subtract(self, a1 , a2, q=0, *waste1, **waste2):
        '''
        function: 区间值广义正交的减法
        :param a1: 一个模糊集, 形式:（[x,y]，[x,y]）
        :param a2: 一个模糊集, 形式:（[x,y]，[x,y]）
        :param q:
        :return: 一个模糊集, 形式:（[x,y]，[x,y]）
        '''
        if q<=0:
            q=self.q
        u1=a1[0][0] * a2[1][0]
        u2=a1[0][1] * a2[1][1]
        v1=pow(pow(a1[1][0], q) + pow(a2[0][0], q) - pow(a1[1][0], q) * pow(a2[0][0], q), 1/q)
        v2=pow(pow(a1[1][1], q) + pow(a2[0][1], q) - pow(a1[1][1], q) * pow(a2[0][1], q), 1/q)

        return ([u1,u2],[v1,v2])

    def divid(self, a1 , a2, q=0, *waste1, **waste2):
        '''
        function: 区间值广义正交的除法
        :param a1: 一个模糊集, 形式:（[x,y]，[x,y]）
        :param a2: 一个模糊集, 形式:（[x,y]，[x,y]）
        :param q:
        :return: 一个模糊集, 形式:（[x,y]，[x,y]）
        '''
        u1= pow(pow(a1[0][0], q) + pow(a2[1][0], q) - pow(a1[0][0], q) * pow(a2[1][0], q), 1/q)
        u2= pow(pow(a1[0][1], q) + pow(a2[1][1], q) - pow(a1[0][1], q) * pow(a2[1][1], q), 1/q)
        v1= a1[1][0] * a2[0][0]
        v2= a1[1][1] * a2[0][1]

        return ([u1,u2],[v1,v2])

    def sortdata(self):
        """
            function : 根据模糊数得分排列
            a        : 一个模糊集, 形式:（[x,y]，[x,y]）
            return   : PWA 集结后的数据结果，形式：（[x,y]，[x,y]）
        """
        s_list = [[self.getsco(ai,self.q).getScore(), ai] for ai in self.data_list]
        s_list.sort(key=lambda x: x[0], reverse=True)
        return [ai[1] for ai in s_list]

In [35]:
class MABAC(Operator_IVQ_ROFS):
    def __init__(self,matrix,weight_list=[],q=3,gangliang=[]):
        '''
        function: MABAC决策方法
        :param matrix: 决策矩阵
        :param weight_list: 属性权重
        :param q: q值
        :param gangliang: 成本型或效益型表示，成本型用0表示，效益型用1表示
        '''
        super(MABAC,self).__init__(q=q)
        self.q=q
        self.matrix = matrix
        if len(weight_list):
            self.weight_list = weight_list
        else:
            self.weight_list = [1/len(matrix[0]) for i in range(len(matrix[0]))]
        # 检测是否传入纲量，若未传入，则全默认为效益型属性
        if len(gangliang):
            self.gangliang=gangliang
        else:
            self.gangliang=[1 for i in range(len(matrix[0]))]
        # 获得方案的最大最小
        self.postive,self.negative=self.get_positive_negative()
        # 获得标准化矩阵
        self.normalizedMatrix=self.getNormalizedMatrix()
        # 获得加权标准化矩阵
        self.weightedNormalizedMatrix=self.getWeightedNormalizedMatrix()
        # 获得边缘向量
        self.borderMatrix=self.getBorderMatrix()
        # 获得与边缘矩阵的距离
        self.distanceBorder=self.getDistanceBorder()
        # 检测是否传入权重，若未传入，则初始值为1/n（n为属性的个数）


    def get_positive_negative(self):
        '''
        function: 获取没个方案中的最大最小值
        :return: 最大最小值的向量
        '''

        score_matrix=[[self.getsco(j,self.q).getScore() for j in i ] for i in self.matrix]
        temp=[[score_matrix[i][j] for i in range(len(score_matrix))]for j in range(len(score_matrix[0]))]
        # 获得最大值与最小值的下标
        max_index=[i.index(max(i)) for i in temp]
        min_index=[i.index(min(i)) for i in temp]
        return max_index,min_index

    def getNormalizedMatrix(self):
        '''
        function: 根据最大最小值，将矩阵中的元素标准化
        :return: 标准化矩阵
        '''
        normalizedMatrix=[]
        for i in range(len(self.matrix)):
            li=[]
            for j in range(len(self.matrix[0])):
                temp_up=self.subtract(self.matrix[i][j],self.matrix[self.negative[j]][j],self.q)
                temp_down=self.subtract(self.matrix[self.postive[j]][j],self.matrix[self.negative[j]][j],self.q)
                temp=self.divid(temp_up,temp_down,self.q)
                li.append(temp)
            normalizedMatrix.append(li)

        return normalizedMatrix

    def getWeightedNormalizedMatrix(self):
        '''
        function：计算加权标准化矩阵，原公式为r_ij=w_j+(r_ij)*(w_j)，现修改为r_ij=(r_ij)*(w_j)
        :return: 加权标准化矩阵
        '''
        weightedNormailedMatrix=[[self.kmulti(self.normalizedMatrix[i][j],self.weight_list[j],self.q)
                                 for j in range(len(self.matrix[0]))] for i in range(len(self.matrix))]

        return weightedNormailedMatrix

    def getBorderMatrix(self):
        '''
        function: 计算边缘向量
        :return: 边缘向量
        '''
        borderMatrix=[]
        for j in range(len(self.matrix[0])):
            temp=self.matrix[0][j]
            for i in range(1,len(self.matrix)):
                temp=self.multi(temp,self.matrix[i][j],self.q)
            temp=self.pow(temp,1/len(self.matrix),self.q)
            borderMatrix.append(temp)

        return borderMatrix

    def getDistanceBorder(self):
        '''
        function: 计算与边缘向量的距离
        :return:
        '''
        distanceMatrix=[]
        for i in range(len(self.weightedNormalizedMatrix)):
            li=[]
            for j in range(len(self.weightedNormalizedMatrix[0])):
                temp=self.subtract(self.weightedNormalizedMatrix[i][j],self.borderMatrix[j],self.q)
                li.append(temp)
            distanceMatrix.append(li)

        return distanceMatrix

    def getResultList(self):
        '''
        function: 计算方案与边缘面积的总近似距离
        :return:
        '''
        totalValue=[]
        for i in range(len(self.matrix)):
            temp=self.distanceBorder[i][0]
            for j in range(1,len(self.matrix[0])):
                temp=self.add(temp,self.distanceBorder[i][j],self.q)
            totalValue.append(temp)

        return totalValue

    def getResult(self):
        '''
        function: 计算方案得分
        :return:
        '''
        value=self.getResultList()
        result=[self.getsco(i,self.q).getScore() for i in value]
        # result=[i/sum(result) for i in result]
        return result